In [3]:
# default_exp to_sagemaker

In [4]:
%load_ext autoreload
%autoreload 2

In [7]:
# export


import os
import subprocess
import sys
from pathlib import Path

from fastcore.script import Param, call_parse
from nbdev.export import find_default_export, get_config, nbglob, read_nb

# Sciflow Notebook to Sagemaker Pipeline

> Converts from a `sciflow` format notebook to a `sagemaker` pipeline. Note: currently this is done via an intermediary step of creating a `metaflow` flow first. *This is temporary and just for convenience as that transformation was developed first.*

Supported features:

* Linear/sequential DAGs
* Simple `Parameters`

In [8]:
nb_path = Path(os.path.join("test", "test_clustering.ipynb"))
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(get_config().path("lib_path"), f"{module_name}.py")